<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-13 15:33:33
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.37 C
-------------------
Today PnL: -2.20 L (-1.58%)
Current PnL: -32.48 L (-20.72%)
CY Booked + Current PnL: -17.74 L (-11.32%)
-------------------
Total profit:  1.27 L
Total loss:  -33.75 L
-------------------
Total Booked + Current PnL: 9.24 L (7.11%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.76%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 97.85 L (71.42%)
Deployed:  1.30 C
Current:  1.37 C
CAGR/XIRR %: 2.77%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,-0.09,2.80,18.25,21.57,18382.0,2745.0,100725.0,397.04,-11.47,61.0,H-SC,3.65,168.0,0.15,0.74,46.99,XR,ATH,FINANCE
77,TTKPRESTIG,-2.36,-26.71,36.56,0.09,27005.0,-26913.0,73864.0,770.00,71.40,41.0,M-SC,8.78,253.0,-1.00,0.54,1.66,OX40N,NTT,DURABLES
18,COALINDIA,-2.43,0.57,18.55,19.23,28068.0,852.0,151312.0,484.83,18.42,41.0,L-LC,11.78,182.0,0.03,1.10,21.96,XY25,ATH,MINING
36,ICICIGI,-0.19,1.50,17.58,19.34,35706.0,2998.0,203107.0,2252.93,-19.79,57.0,X-MC,3.27,71.0,0.08,1.48,17.67,X40,ATH,INSURANCE
51,MEDANTA,-0.83,-4.89,30.45,24.08,36767.0,-6205.0,120745.0,1486.00,-12.55,52.0,X-SC,11.72,91.0,-0.17,0.88,10.59,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,5.13,-43.91,141.18,35.27,89912.0,-49863.0,63686.0,80.0,-57.75,54.0,L-SC,14.38,268.0,-0.55,0.46,55.87,XR,NTT,HOTELS
83,WHIRLPOOL,1.13,-28.00,152.67,81.92,126186.0,-32145.0,82653.0,2270.0,-57.77,60.0,M-SC,30.80,218.0,-0.25,0.60,16.10,XR,NTT,DURABLES
37,ICICIPRULI,0.87,6.51,22.74,30.73,47715.0,12825.0,209830.0,790.0,-18.40,46.0,X-MC,3.92,79.0,0.27,1.53,20.11,X40,ATH,INSURANCE
41,INDUSINDBK,0.36,-31.63,94.37,32.89,49813.0,-24422.0,52785.0,1800.0,-752.02,58.0,L-MC,11.20,259.0,-0.49,0.39,45.37,XR,NTT,BANKS
13,BERGEPAINT,0.27,-17.70,47.19,21.14,88297.0,-40229.0,187110.0,680.0,-23.58,30.0,X-MC,11.05,75.0,-0.46,1.37,0.76,XY24,NTT,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BSOFT,-8.04,-27.67,108.47,50.79,110349.0,-38921.0,101732.0,831.7,-7.23,41.0,H-SC,7.73,171.0,-0.35,0.74,18.45,XR,ATH,IT
64,SHALBY,-6.89,-40.02,108.59,25.10,117974.0,-72501.0,108642.0,327.0,699.04,32.0,M-SC,16.18,248.0,-0.61,0.79,0.00,XY24,NTT,HEALTHCARE
70,SYMPHONY,-6.10,-32.84,48.86,-0.03,56156.0,-56208.0,114933.0,1306.0,-46.08,43.0,M-SC,10.38,206.0,-1.00,0.84,6.21,OX40N,NTT,DURABLES
34,HINDUNILVR,-4.34,-10.33,26.76,13.67,61687.0,-26543.0,230520.0,2922.0,-17.11,39.0,X-LC,2.73,9.0,-0.43,1.68,8.83,XY25,NTT,FMCG
66,SIS,-3.82,-24.82,61.30,21.26,51170.0,-27558.0,83474.0,528.0,1968.39,46.0,H-SC,4.05,163.0,-0.54,0.61,15.08,OX40N,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-2.43,0.57,18.55,19.23,28068.0,852.0,151312.0,484.83,18.42,41.0,L-LC,11.78,182.0,0.03,1.1,21.96,XY25,ATH,MINING


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,-1.01,2.69,102.46,107.91,155350.0,3976.0,151620.0,1641.55,-16.50,64.0,M-SC,9.98,220.0,0.03,1.11,14.07,OX40N,ATH,CABLES
66,SIS,-3.82,-24.82,61.30,21.26,51170.0,-27558.0,83474.0,528.00,1968.39,46.0,H-SC,4.05,163.0,-0.54,0.61,15.08,OX40N,NTT,MISC
77,TTKPRESTIG,-2.36,-26.71,36.56,0.09,27005.0,-26913.0,73864.0,770.00,71.40,41.0,M-SC,8.78,253.0,-1.00,0.54,1.66,OX40N,NTT,DURABLES
72,TATAELXSI,-3.05,-30.48,89.90,32.03,82399.0,-40179.0,91656.0,9161.00,-24.77,32.0,H-SC,6.19,157.0,-0.49,0.67,2.66,OX40N,NTT,IT
40,INDIGOPNTS,-2.44,-30.87,44.69,0.02,53925.0,-53895.0,120664.0,1408.00,84.15,28.0,M-SC,7.45,240.0,-1.00,0.88,4.96,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-2.43,0.57,18.55,19.23,28068.0,852.0,151312.0,484.83,18.42,41.0,L-LC,11.78,182.0,0.03,1.10,21.96,XY25,ATH,MINING
50,MASFIN,-0.09,2.80,18.25,21.57,18382.0,2745.0,100725.0,397.04,-11.47,61.0,H-SC,3.65,168.0,0.15,0.74,46.99,XR,ATH,FINANCE
26,FINCABLES,-1.01,2.69,102.46,107.91,155350.0,3976.0,151620.0,1641.55,-16.50,64.0,M-SC,9.98,220.0,0.03,1.11,14.07,OX40N,ATH,CABLES
1,ABB,-0.14,6.06,37.25,45.57,103355.0,15845.0,277464.0,7934.00,-31.89,68.0,H-MC,2.97,121.0,0.15,2.03,23.21,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,-0.14,6.06,37.25,45.57,103355.0,15845.0,277464.0,7934.00,-31.89,68.0,H-MC,2.97,121.0,0.15,2.03,23.21,AR,NTT,ELECTRICAL
50,MASFIN,-0.09,2.80,18.25,21.57,18382.0,2745.0,100725.0,397.04,-11.47,61.0,H-SC,3.65,168.0,0.15,0.74,46.99,XR,ATH,FINANCE
26,FINCABLES,-1.01,2.69,102.46,107.91,155350.0,3976.0,151620.0,1641.55,-16.50,64.0,M-SC,9.98,220.0,0.03,1.11,14.07,OX40N,ATH,CABLES
39,INDIAMART,-0.01,-5.46,118.66,106.72,138358.0,-6736.0,116600.0,4810.62,-53.11,47.0,H-SC,7.62,138.0,-0.05,0.85,16.64,AR,ATH,MISC
85,ZYDUSLIFE,-1.76,-4.60,42.17,35.63,84314.0,-9635.0,199939.0,1286.17,-18.07,51.0,H-MC,5.96,119.0,-0.11,1.46,11.31,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,-1.22,-6.80,44.04,34.24,128429.0,-21270.0,291618.0,1972.00,-27.49,20.0,X-LC,4.79,6.0,-0.17,2.13,0.92,X40,NTT,IT
33,HCLTECH,-1.42,-5.56,31.13,23.83,71121.0,-13458.0,228466.0,1908.19,-1.23,22.0,X-LC,5.30,13.0,-0.19,1.67,9.61,X40,ATH,IT
8,AWL,-2.29,-34.67,140.61,57.19,299582.0,-113068.0,213059.0,485.00,-69.12,25.0,X-MC,17.42,58.0,-0.38,1.56,0.00,XY24,NTT,FMCG
73,TCS,-2.11,-25.98,60.15,18.55,168413.0,-98248.0,279989.0,4311.59,-36.25,25.0,X-LC,2.40,3.0,-0.58,2.04,0.00,X40,ATH,IT
6,ASIANPAINT,-1.83,-17.29,46.22,20.95,96250.0,-43525.0,208243.0,3460.25,-18.06,28.0,X-LC,7.17,36.0,-0.45,1.52,12.19,X40,ATH,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,DABUR,-1.37,0.19,43.43,43.70,107940.0,470.0,248538.0,735.00,-8.14,51.0,X-MC,1.80,73.0,0.00,1.81,15.27,XY24,BTT,FMCG
73,TCS,-2.11,-25.98,60.15,18.55,168413.0,-98248.0,279989.0,4311.59,-36.25,25.0,X-LC,2.40,3.0,-0.58,2.04,0.00,X40,ATH,IT
81,VBL,-1.40,-9.24,49.49,35.67,141865.0,-29189.0,286653.0,671.64,-19.99,38.0,X-LC,2.44,5.0,-0.21,2.09,3.33,X40N,ATH,FMCG
78,UNITDSPR,-1.06,3.02,17.23,20.77,41561.0,7069.0,241213.0,1644.00,-5.95,59.0,X-MC,2.59,70.0,0.17,1.76,10.64,X40N,NTT,BREWERIES
65,SIEMENS,-1.02,-16.27,49.90,25.51,77757.0,-30270.0,155825.0,4671.50,-4.35,52.0,H-LC,2.68,50.0,-0.39,1.14,14.83,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
2,ABBOTINDIA,-0.75,-11.24,33.01,18.07,52407.0,-20095.0,158760.0,35195.00,-23.91,35.0,X-MC,10.13,64.0,-0.38,1.16,0.00,X40,ATH,PHARMA
8,AWL,-2.29,-34.67,140.61,57.19,299582.0,-113068.0,213059.0,485.00,-69.12,25.0,X-MC,17.42,58.0,-0.38,1.56,0.00,XY24,NTT,FMCG
73,TCS,-2.11,-25.98,60.15,18.55,168413.0,-98248.0,279989.0,4311.59,-36.25,25.0,X-LC,2.40,3.0,-0.58,2.04,0.00,X40,ATH,IT
3,ACC,-2.05,-31.20,138.59,64.15,226886.0,-74241.0,163710.0,3906.00,-59.81,38.0,X-SC,7.93,84.0,-0.33,1.20,0.07,XY24,BTT,CEMENT
12,BATAINDIA,-3.57,-45.02,145.94,35.20,103234.0,-57933.0,70737.0,2096.00,-7.29,39.0,X-SC,21.05,93.0,-0.56,0.52,0.52,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,-0.63,-30.90,106.71,42.84,47935.0,-20085.0,44921.0,424.00,-54.82,49.0,X-SC,20.00,83.0,-0.42,0.33,8.05,XY24,NTT,MISC
58,RELAXO,-1.86,-53.40,226.85,52.31,153446.0,-77518.0,67642.0,1176.00,-50.34,36.0,X-SC,13.83,92.0,-0.51,0.49,0.59,X40N,NTT,FOOTWEAR
12,BATAINDIA,-3.57,-45.02,145.94,35.20,103234.0,-57933.0,70737.0,2096.00,-7.29,39.0,X-SC,21.05,93.0,-0.56,0.52,0.52,X40,NTT,FOOTWEAR
51,MEDANTA,-0.83,-4.89,30.45,24.08,36767.0,-6205.0,120745.0,1486.00,-12.55,52.0,X-SC,11.72,91.0,-0.17,0.88,10.59,XY24,NTT,HEALTHCARE
35,HONAUT,-1.54,-22.95,86.00,43.32,107930.0,-37372.0,125500.0,58357.33,-33.73,38.0,X-SC,10.28,90.0,-0.35,0.92,1.90,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-1.17,-23.10,44.06,10.78,81422.0,-55519.0,184799.0,452.00,-54.23,37.0,X-LC,18.96,1.0,-0.68,1.35,1.79,X40,NTT,FMCG
75,TMPV,-0.83,-22.06,57.79,22.98,101743.0,-49842.0,176056.0,600.00,-19.41,63.0,X-LC,3.36,2.0,-0.49,1.29,12.55,XY24,NTT,AUTO
73,TCS,-2.11,-25.98,60.15,18.55,168413.0,-98248.0,279989.0,4311.59,-36.25,25.0,X-LC,2.40,3.0,-0.58,2.04,0.00,X40,ATH,IT
81,VBL,-1.40,-9.24,49.49,35.67,141865.0,-29189.0,286653.0,671.64,-19.99,38.0,X-LC,2.44,5.0,-0.21,2.09,3.33,X40N,ATH,FMCG
42,INFY,-1.22,-6.80,44.04,34.24,128429.0,-21270.0,291618.0,1972.00,-27.49,20.0,X-LC,4.79,6.0,-0.17,2.13,0.92,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,0.36,-31.63,94.37,32.89,49813.0,-24422.0,52785.0,1800.00,-752.02,58.0,L-MC,11.20,259.0,-0.49,0.39,45.37,XR,NTT,BANKS
67,SONACOMS,-0.91,-8.67,52.29,39.09,48311.0,-8768.0,92391.0,804.02,-28.63,63.0,M-SC,4.43,227.0,-0.18,0.67,30.70,AR,ATH,AUTO
50,MASFIN,-0.09,2.80,18.25,21.57,18382.0,2745.0,100725.0,397.04,-11.47,61.0,H-SC,3.65,168.0,0.15,0.74,46.99,XR,ATH,FINANCE
79,VAIBHAVGBL,-1.42,-21.86,105.75,60.77,151028.0,-39959.0,142816.0,521.00,64.74,60.0,H-SC,1.44,158.0,-0.26,1.04,29.48,XR,NTT,JEWELLERY
18,COALINDIA,-2.43,0.57,18.55,19.23,28068.0,852.0,151312.0,484.83,18.42,41.0,L-LC,11.78,182.0,0.03,1.10,21.96,XY25,ATH,MINING


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,-0.14,6.06,37.25,45.57,103355.0,15845.0,277464.0,7934.00,-31.89,68.0,H-MC,2.97,121.0,0.15,2.03,23.21,AR,NTT,ELECTRICAL
67,SONACOMS,-0.91,-8.67,52.29,39.09,48311.0,-8768.0,92391.0,804.02,-28.63,63.0,M-SC,4.43,227.0,-0.18,0.67,30.70,AR,ATH,AUTO
11,BANDHANBNK,-2.42,-20.42,141.78,92.42,314312.0,-56869.0,221690.0,400.00,55.89,66.0,H-SC,8.10,169.0,-0.18,1.62,28.12,XY24,NTT,BANKS
7,ATULAUTO,-2.32,-13.70,68.68,45.57,116838.0,-27013.0,170119.0,844.00,3689.19,66.0,M-SC,5.83,245.0,-0.23,1.24,28.74,XY24,NTT,AUTO
79,VAIBHAVGBL,-1.42,-21.86,105.75,60.77,151028.0,-39959.0,142816.0,521.00,64.74,60.0,H-SC,1.44,158.0,-0.26,1.04,29.48,XR,NTT,JEWELLERY


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.26
1,25,44.65
2,50,76.70


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.57
MC    31.25
LC    23.16
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.68
X40      23.13
X40N     15.28
AR        9.00
XR        8.89
XY25      8.73
OX40N     7.39
SR        0.88
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.51
H-SC    22.84
X-LC    19.83
M-SC    11.52
X-SC    10.41
H-MC     4.96
M-MC     1.39
H-LC     1.14
L-LC     1.10
M-LC     1.09
L-SC     0.80
L-MC     0.39
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.99,-15.28,54.01
FINANCE,13.48,-13.04,57.52
IT,11.55,-35.04,102.43
MISC,7.11,-33.59,88.23
ELECTRICAL,6.24,-9.95,50.07
PAINTS,5.08,-32.65,52.36
INSURANCE,4.87,-1.62,36.39
PHARMA,4.14,-3.79,36.46
AUTO,3.20,-19.52,60.86


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3315848.0,21
AR,1339728.0,10
X40,1328360.0,15
XR,1273219.0,12
X40N,1027349.0,10
OX40N,779025.0,10
XY25,422981.0,6
SR,298300.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3662851.0,24
X-MC,1669687.0,16
M-SC,1477515.0,15
X-LC,1175062.0,12
X-SC,866339.0,9
H-MC,400066.0,3
L-SC,180033.0,2
M-LC,143109.0,1
H-LC,77757.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1255470.0      6
           AR         951477.0      5
M-SC       XY24       833005.0      6
H-SC       XR         811538.0      7
X-LC       X40        606326.0      6
X-MC       X40        546031.0      7
           XY24       495819.0      3
           X40N       417794.0      4
X-SC       X40N       378748.0      4
H-SC       OX40N      346066.0      4
M-SC       OX40N      342838.0      5
X-SC       XY24       311588.0      3
H-SC       SR         298300.0      2
X-LC       X40N       230807.0      2
H-MC       XY24       212397.0      1
X-MC       XY25       210043.0      2
X-LC       XY24       207569.0      2
H-MC       AR         187669.0      2
M-SC       XR         178847.0      2
X-SC       X40        176003.0      2
M-LC       XR         143109.0      1
X-LC       XY25       130360.0      2
M-SC       AR         122825.0      2
L-SC       OX40N       90121.0      1
           XR          89912.0      1
H-LC       AR          77757.0      1
M-MC       XY25        54510.0      1
L-MC       XR          49813.0      1
L-LC       XY25        28068.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
